# ML testing: experiment #6

This notebook involves testing for the MRI conference abstract. This notebook shows mixed_dataset (TOP + StrokeMRI)  based models with harmonization, applied to TOP, MRI and Sabre datasets by neurocombat. Each was considered a seperate batch. Nota Bene: The mixed dataset for the model did not include SABRE

### import libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import sys

import glob
from functools import reduce

import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt

# # demo stuff
# import ipywidgets as widgets
# import seaborn 

# ml stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import tree
from sklearn import metrics
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import ExtraTreesRegressor

import joblib

sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config

### import data

In [ ]:
)
filepath_mri = '../open_work/internal_results/harmonized_pvc2s/three_harm/' 
filename_mri = os.path.join(filepath_mri,'neuro_harm3way_mri.csv') 
filepath_top = '../open_work/internal_results/harmonized_pvc2s/three_harm/' 
filename_top = os.path.join(filepath_top,'neuro_harm3way_top.csv') 
filepath_sabre = '../open_work/internal_results/harmonized_pvc2s/three_harm/' 
filename_sabre = os.path.join(filepath_top,'neuro_harm3way_sabre.csv') 
# read in data
TOP = pd.read_csv(filename_top)
StrokeMRI = pd.read_csv(filename_mri)
SABRE = pd.read_csv(filename_sabre)
# take extra column off
TOP = TOP.drop(TOP.columns[0],axis=1)
SABRE = SABRE.drop(SABRE.columns[0],axis=1)
StrokeMRI = StrokeMRI.drop(StrokeMRI.columns[0],axis=1)

In [ ]:
# Now we need to flip the sex back to numbers for a correlation
sex_mapping = {'F':0,'M':1}
TOP = TOP.assign(sex = TOP.sex.map(sex_mapping))
TOP.head(3)

In [ ]:
StrokeMRI = StrokeMRI.assign(sex = StrokeMRI.sex.map(sex_mapping))
StrokeMRI.head(3)

In [ ]:
SABRE = SABRE.assign(sex = SABRE.sex.map(sex_mapping))
SABRE.head(3)

In [ ]:
#SABRE =SABRE.dropna()
SABRE.isna().sum()

In [ ]:
# check for any duplicated patients between stroke and mri
strokers = set(StrokeMRI.participant_id)
topers = set(TOP.participant_id)
z = strokers.intersection(topers)
print(z) 

In [ ]:
# make mixed StrokeMRI and TOP dataset
mixed_data = pd.concat([TOP, StrokeMRI], sort=False)


## Build ML models

# keeping patient ID until right when model is fed, then use patient ID as key to what went where

In [ ]:
ml_matrix = mixed_data #.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
y = ml_matrix['age'].values
y=y.astype('float')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12)

In [ ]:
X_train_cut = X_train[:,1:]
X_train_cut = X_train_cut.astype('float')
X_train_cut.shape

In [ ]:
X_test_cut = X_test[:,1:]
X_test_cut = X_test_cut.astype('float')
X_test_cut.shape

In [ ]:

# regr = MLPRegressor(random_state=1, max_iter=700)
# regr.fit(X_train_cut, y_train)

In [ ]:
# y_pred = regr.predict(X_test_cut)

In [ ]:
# print('R2 score neural network mlp regression: %.3f' % regr.score(X_test_cut,y_test))
# print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
# print('MAE: % .3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
linr = LinearRegression()
linr.fit(X_train_cut, y_train)

In [ ]:
y_pred = linr.predict(X_test_cut)

In [ ]:
print('R2 score Linear regression: %.3f' % linr.score(X_test_cut,y_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
print('MAE: % .3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
data= [[
    'linear regression',
    'unharm_mixed_linr.sav',
    mean_absolute_error(y_test, y_pred),
    linr.score(X_test_cut,y_test),
    metrics.explained_variance_score(y_test, y_pred)]]
linr_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#linr_results

In [ ]:
linr_y_test = y_test
linr_y_pred = y_pred
linr_compare = pd.DataFrame(
    {'y_test_real_age': linr_y_test,
     'linr_y_pred_age': linr_y_pred,
    })
linr_compare = linr_compare.reset_index()
#linr_compare

In [ ]:
llreg = linear_model.LassoLars(alpha=0.01)
llreg.fit(X_train_cut, y_train)

In [ ]:
y_pred = llreg.predict(X_test_cut)
print('R2 score Lasso regression: %.3f' % llreg.score(X_test_cut,y_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
data= [[
    'lasso linear regression',
    'unharm_mixed_lassor.sav',
    mean_absolute_error(y_test, y_pred),
    llreg.score(X_test_cut,y_test),
    metrics.explained_variance_score(y_test, y_pred)]]
llreg_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#llreg_results

In [ ]:
llreg_y_test = y_test
llreg_y_pred = y_pred
llreg_compare = pd.DataFrame(
    {'y_test_real_age': llreg_y_test,
     'lasso_y_pred_age': llreg_y_pred,
    })
llreg_compare = llreg_compare.reset_index()
#llreg_compare

In [ ]:
dtree = tree.DecisionTreeRegressor()
dtree.fit(X_train_cut, y_train)

In [ ]:
y_pred = dtree.predict(X_test_cut)
print('R2 score dtree regression: %.3f' % dtree.score(X_test_cut,y_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
data= [[
    'decision tree',
    'unharm_mixed_dtree.sav',
    mean_absolute_error(y_test, y_pred),
    dtree.score(X_test_cut,y_test),
    metrics.explained_variance_score(y_test, y_pred)]]
dtree_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#dtree_results

In [ ]:
dtree_y_test = y_test
dtree_y_pred = y_pred
dtree_compare = pd.DataFrame(
    {'y_test_real_age': dtree_y_test,
     'dtree_y_pred_age': dtree_y_pred,
    })
dtree_compare = dtree_compare.reset_index()
#dtree_compare

In [ ]:
regr = MLPRegressor(random_state=1, max_iter=700)
regr.fit(X_train_cut, y_train)

In [ ]:
y_pred = regr.predict(X_test_cut)

In [ ]:
# print('R2 score neural network mlp regression: %.3f' % regr.score(X_test_cut,y_test))
# print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
# print('MAE: % .3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
data= [[
    'MLP regression',
    'unharm_mixed_regr.sav',
    mean_absolute_error(y_test, y_pred),
    regr.score(X_test_cut,y_test),
    metrics.explained_variance_score(y_test, y_pred)]]
regr_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
regr_results

In [ ]:
regr_y_test = y_test
regr_y_pred = y_pred
regr_compare = pd.DataFrame(
    {'y_test_real_age': regr_y_test,
     'regr_y_pred_age': regr_y_pred,
    })
regr_compare = regr_compare.reset_index()
#regr_compare

In [ ]:
svr_p2 = SVR(C=1.0, kernel='poly', degree =2, epsilon=0.2)
svr_p2.fit(X_train_cut, y_train)

In [ ]:
y_pred = svr_p2.predict(X_test_cut)

In [ ]:
# print('R2 score SVR 2nd degree poly kernel regression: %.3f' % svr_p2.score(X_test_cut,y_test))
# print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
# print('MAE: % .3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
data= [[
    'SVR poly2',
    'unharm_mixed_svrp2.sav',
    mean_absolute_error(y_test, y_pred),
    svr_p2.score(X_test_cut,y_test),
    metrics.explained_variance_score(y_test, y_pred)]]
svr_p2_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#svr_p2_results

In [ ]:
svrp2_y_test = y_test
svrp2_y_pred = y_pred
svrp2_compare = pd.DataFrame(
    {'y_test_real_age': svrp2_y_test,
     'svrp2_y_pred_age': svrp2_y_pred,
    })
svrp2_compare = svrp2_compare.reset_index()
#svrp2_compare

In [ ]:
eregr = ElasticNetCV(cv=5, random_state=12)
eregr.fit(X_train_cut, y_train)

In [ ]:
y_pred = eregr.predict(X_test_cut)

In [ ]:
# print('R2 score elasticnetcv regression: %.3f' % eregr.score(X_test_cut,y_test))
# print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
# print('MAE: % .3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
data= [[
    'Elastic_netCV',
    'unharm_mixed_eregr.sav',
    mean_absolute_error(y_test, y_pred),
    eregr.score(X_test_cut,y_test),
    metrics.explained_variance_score(y_test, y_pred)]]
eregr_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#eregr_results

In [ ]:
eregr_y_test = y_test
eregr_y_pred = y_pred
eregr_compare = pd.DataFrame(
    {'y_test_real_age': eregr_y_test,
     'eregr_y_pred_age': eregr_y_pred,
    })
eregr_compare = eregr_compare.reset_index()
#eregr_compare

In [ ]:
etreg = ExtraTreesRegressor(n_estimators=100, random_state=0)
etreg.fit(X_train_cut, y_train)

In [ ]:
y_pred = etreg.predict(X_test_cut)

In [ ]:
# print('R2 score extra trees regression: %.3f' % etreg.score(X_test_cut,y_test))
# print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_test, y_pred))
# print('MAE: % .3f' % mean_absolute_error(y_test, y_pred))

In [ ]:
data= [[
    'Extra trees',
    'unharm_mixed_etreg.sav',
    mean_absolute_error(y_test, y_pred),
    etreg.score(X_test_cut,y_test),
    metrics.explained_variance_score(y_test, y_pred)]]
etreg_results = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#etreg_results

In [ ]:
etreg_y_test = y_test
etreg_y_pred = y_pred
etreg_compare = pd.DataFrame(
    {'y_test_real_age': etreg_y_test,
     'etreg_y_pred_age': etreg_y_pred,
    })
etreg_compare = etreg_compare.reset_index()
#etreg_compare

Show results

In [ ]:
mixed_based_unharmonized_on_testmix =pd.concat([linr_results,
                   llreg_results,
                   dtree_results,
                   regr_results,
                   svr_p2_results,
                   eregr_results,
                  etreg_results],
                  axis=0)
mixed_based_unharmonized_on_testmix

In [ ]:
data_frames1 = [linr_compare, llreg_compare, dtree_compare,]# regr_compare, ]#etreg_compare, svrp2_compare,]
real_versus_projected_y1 = reduce(lambda  left,right: pd.merge(left,right,on=["index"],
                                            how='outer'), data_frames1)
#real_versus_projected_y1
data_frames2 = [eregr_compare, svrp2_compare, etreg_compare,]
real_versus_projected_y2 = reduce(lambda  left,right: pd.merge(left,right,on=["index"],
                                            how='outer'), data_frames2)
#real_versus_projected_y2
real_versus_projected_y2 = sep.drop_y(real_versus_projected_y2 )
real_versus_projected_y1 = sep.drop_y(real_versus_projected_y1 )
#real_versus_projected_y1

In [ ]:
data_frames3 = [real_versus_projected_y1, real_versus_projected_y2,]
real_versus_projected_y3_mixed_on_mixed = reduce(lambda  left,right: pd.merge(left,right,on=["index"],
                                            how='outer'), data_frames3)
real_versus_projected_y3_mixed_on_mixed.head(3)

## Save off models and csv (optional, must uncomment)

In [ ]:
# # optionally save of csvs of algorithms and results
# mixed_based_unharmonized_on_testmix.to_csv('mixed_based_unharmonized_on_testmix.csv')
# real_versus_projected_y3_mixed_on_mixed.to_csv('real_versus_projected_y3_mixed_on_mixed.csv')

In [ ]:
# check if model folder exists and if not , then create
model_folder = '../result_models/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [ ]:
# joblib.dump(linr, ('../result_models/'+ 'unharm_mixed_linr.sav'))
# joblib.dump(llreg, ('../result_models/'+ 'unharm_mixed_lassor.sav'))
# joblib.dump(dtree, ('../result_models/'+ 'unharm_mixed_dtree.sav'))
# joblib.dump(regr, ('../result_models/'+'unharm_mixed_regr.sav'))
# joblib.dump(svr_p2, ('../result_models/'+'unharm_mixed_svrp2.sav'))
# joblib.dump(eregr, ('../result_models/'+ 'unharm_mixed_elasticnet.sav'))
# joblib.dump(etreg, ('../result_models/'+ 'unharm_mixed_extratree.sav'))

## Run models on other datasets (TOP, StrokeMRI)
but without re-running the training data

# # Here we check that no rows are duplicated once patient IDs were pulled 
(if not we can map them back)

In [ ]:
X_train_pandas = pd.DataFrame(X_train)
X_train_pandas.duplicated().sum()

top_ml_matrix
needs to be mapped to top rows in X_train,
we will use ese MD5 hashes

now we need to make a dataframe of TOP minus what is in X_train

In [ ]:
X_train_pandas.head(3)

In [ ]:
trained_subjects = set(X_train_pandas[0])
#trained_subjects 

In [ ]:
TOP_subjects = set(TOP.participant_id)
#TOP_subjects

In [ ]:
# take trained subjects out of top subjects
# we can use set math here
new_top=(trained_subjects^TOP_subjects)&TOP_subjects
print(len(new_top))
#print(new_top)

filter down to only top where they are in new_top set

In [ ]:
TOP_new = TOP[TOP['participant_id'].isin(list(new_top))]
#TOP_new

In [ ]:
top_ml_matrix = TOP_new.drop('participant_id', axis=1) 

X_top = top_ml_matrix.drop('age', axis =1)

X_top = X_top.values
X_top = X_top.astype('float')
y_top = top_ml_matrix['age'].values
y_top=y_top.astype('float')

In [ ]:
X_top_test = X_top
y_top_test = y_top

In [ ]:
y_top_pred = linr.predict(X_top_test)

In [ ]:
print('R2 score Linear regression: %.3f' % linr.score(X_top_test,y_top_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_top_test, y_top_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_top_test, y_top_pred))

In [ ]:
data= [[
    'linear regression',
    'unharm_mixed_linr.sav',
    mean_absolute_error(y_top_test, y_top_pred),
    linr.score(X_top_test,y_top_test),
    metrics.explained_variance_score(y_top_test, y_top_pred)]]
linr_results_top = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_top

In [ ]:
linr_y_test = y_top_test
linr_y_pred = y_top_pred
linr_compare = pd.DataFrame(
    {'y_test_real_age': linr_y_test,
     'linr_y_pred_age': linr_y_pred,
    })
linr_compare = linr_compare.reset_index()
#linr_compare

In [ ]:
y_top_pred = llreg.predict(X_top_test)

In [ ]:
print('R2 score Lasso linear regression: %.3f' % llreg.score(X_top_test,y_top_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_top_test, y_top_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_top_test, y_top_pred))

In [ ]:
data= [[
    'lasso regression',
    'unharm_mixed_llregr.sav',
    mean_absolute_error(y_top_test, y_top_pred),
    llreg.score(X_top_test,y_top_test),
    metrics.explained_variance_score(y_top_test, y_top_pred)]]
llreg_results_top = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_top

In [ ]:
llreg_y_test = y_top_test
llreg_y_pred = y_top_pred
llreg_compare = pd.DataFrame(
    {'y_test_real_age': llreg_y_test,
     'linr_y_pred_age': llreg_y_pred,
    })
llreg_compare = llreg_compare.reset_index()
#llreg_compare

In [ ]:
y_top_pred = dtree.predict(X_top_test)

In [ ]:
print('R2 score decision tree regression: %.3f' % dtree.score(X_top_test,y_top_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_top_test, y_top_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_top_test, y_top_pred))

In [ ]:
data= [[
    'decision tree',
    'unharm_mixed_dtree.sav',
    mean_absolute_error(y_top_test, y_top_pred),
    dtree.score(X_top_test,y_top_test),
    metrics.explained_variance_score(y_top_test, y_top_pred)]]
dtree_results_top = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
dtree_results_top

In [ ]:
dtree_y_test = y_top_test
dtree_y_pred = y_top_pred
dtree_compare = pd.DataFrame(
    {'y_test_real_age': dtree_y_test,
     'linr_y_pred_age': dtree_y_pred,
    })
dtree_compare = linr_compare.reset_index()
#dtree_compare

In [ ]:
y_top_pred = eregr.predict(X_top_test)

In [ ]:
print('R2 score ElasticnetCV regression: %.3f' % eregr.score(X_top_test,y_top_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_top_test, y_top_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_top_test, y_top_pred))

In [ ]:
data= [[
    'elasticnetCV',
    'unharm_mixed_eregr.sav',
    mean_absolute_error(y_top_test, y_top_pred),
    linr.score(X_top_test,y_top_test),
    metrics.explained_variance_score(y_top_test, y_top_pred)]]
eregr_results_top = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_top

In [ ]:
eregr_y_test = y_top_test
eregr_y_pred = y_top_pred
eregr_compare = pd.DataFrame(
    {'y_test_real_age': eregr_y_test,
     'linr_y_pred_age': eregr_y_pred,
    })
eregr_compare = linr_compare.reset_index()
#eregr_compare

In [ ]:
y_top_pred = svr_p2.predict(X_top_test)

In [ ]:
print('R2 score SVR poly 2 regression: %.3f' % svr_p2.score(X_top_test,y_top_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_top_test, y_top_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_top_test, y_top_pred))

In [ ]:
data= [[
    'SVR polynom degree 2',
    'unharm_mixed_svrp2.sav',
    mean_absolute_error(y_top_test, y_top_pred),
    svr_p2.score(X_top_test,y_top_test),
    metrics.explained_variance_score(y_top_test, y_top_pred)]]
svrp2_results_top = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#svrp2_results_top

In [ ]:
svrp2_y_test = y_top_test
svrp2_y_pred = y_top_pred
svrp2_compare = pd.DataFrame(
    {'y_test_real_age': svrp2_y_test,
     'linr_y_pred_age': svrp2_y_pred,
    })
svrp2_compare = linr_compare.reset_index()
#svrp2_compare

In [ ]:
y_top_pred = etreg.predict(X_top_test)

In [ ]:
print('R2 score Extra trees: %.3f' % etreg.score(X_top_test,y_top_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_top_test, y_top_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_top_test, y_top_pred))

In [ ]:
data= [[
    'extra trees',
    'unharm_mixed_ereg.sav',
    mean_absolute_error(y_top_test, y_top_pred),
    etreg.score(X_top_test,y_top_test),
    metrics.explained_variance_score(y_top_test, y_top_pred)]]
etreg_results_top = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_top

In [ ]:
etreg_y_test = y_top_test
etreg_y_pred = y_top_pred
etreg_compare = pd.DataFrame(
    {'y_test_real_age': etreg_y_test,
     'linr_y_pred_age': etreg_y_pred,
    })
etreg_compare = linr_compare.reset_index()
#etreg_compare

In [ ]:
y_top_pred = regr.predict(X_top_test)

In [ ]:
print('R2 score MLP regression: %.3f' % regr.score(X_top_test,y_top_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_top_test, y_top_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_top_test, y_top_pred))

In [ ]:
data= [[
    'multilayered percentron',
    'unharm_mixed_regr.sav',
    mean_absolute_error(y_top_test, y_top_pred),
    regr.score(X_top_test,y_top_test),
    metrics.explained_variance_score(y_top_test, y_top_pred)]]
regr_results_top = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#regr_results_top

In [ ]:
regr_y_test = y_top_test
regr_y_pred = y_top_pred
regr_compare = pd.DataFrame(
    {'y_test_real_age': regr_y_test,
     'linr_y_pred_age': regr_y_pred,
    })
regr_compare = linr_compare.reset_index()
#regr_compare

In [ ]:
# Compile results of mixed on TOP

In [ ]:
mixed_based_unharmonized_on_top =pd.concat([linr_results_top,
                   llreg_results_top,
                   dtree_results_top,
                   regr_results_top,
                   svrp2_results_top,
                   eregr_results_top,
                  etreg_results_top],
                  axis=0)
mixed_based_unharmonized_on_top

In [ ]:
data_frames1 = [linr_compare, llreg_compare, dtree_compare,]# regr_compare, ]#etreg_compare, svrp2_compare,]
real_versus_projected_y1 = reduce(lambda  left,right: pd.merge(left,right,on=["index"],
                                            how='outer'), data_frames1)
#real_versus_projected_y1
data_frames2 = [eregr_compare, svrp2_compare, etreg_compare,]
real_versus_projected_y2 = reduce(lambda  left,right: pd.merge(left,right,on=["index"],
                                            how='outer'), data_frames2)
#real_versus_projected_y2
real_versus_projected_y2 = sep.drop_y(real_versus_projected_y2 )
real_versus_projected_y1 = sep.drop_y(real_versus_projected_y1 )
#real_versus_projected_y1

In [ ]:
data_frames3 = [real_versus_projected_y1, real_versus_projected_y2,]
real_versus_projected_y3_mixed_on_top = reduce(lambda  left,right: pd.merge(left,right,on=["index"],
                                            how='outer'), data_frames3)
real_versus_projected_y3_mixed_on_top.head(3)

## Save off csvs of results 
optional, you must uncomment

In [ ]:
## optionally save of csvs of algorithms and results
#mixed_based_unharmonized_on_top.to_csv('mixed_based_unharmonized_on_top.csv')
#real_versus_projected_y3_mixed_on_top.to_csv('real_versus_projected_y3_mixed_on_top.csv')

# Mixed algorithm on stroke MRI subjects (not in training)

In [ ]:
StrokeMRI_subjects = set(StrokeMRI.participant_id)
#StrokeMRI_subjects

In [ ]:
# take trained subjects out of top subjects
# we can use set math here
new_mri=(trained_subjects^StrokeMRI_subjects)&StrokeMRI_subjects

In [ ]:
StrokeMRI_new = StrokeMRI[StrokeMRI['participant_id'].isin(list(new_mri))]
#StrokeMRI_new

In [ ]:
strokemri_ml_matrix = StrokeMRI_new.drop('participant_id', axis=1) 
X_mri = strokemri_ml_matrix.drop('age', axis =1)
X_mri = X_mri.values
X_mri = X_mri.astype('float')
y_mri = strokemri_ml_matrix['age'].values
y_mri=y_mri.astype('float')

In [ ]:
X_mri_test = X_mri
y_mri_test = y_mri

In [ ]:
y_mri_pred = linr.predict(X_mri_test)

In [ ]:
print('R2 score Linear regression: %.3f' % linr.score(X_mri_test,y_mri_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_mri_test, y_mri_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_mri_test, y_mri_pred))

In [ ]:
data= [[
    'linear regression',
    'unharm_mixed_linr.sav',
    mean_absolute_error(y_mri_test, y_mri_pred),
    linr.score(X_mri_test,y_mri_test),
    metrics.explained_variance_score(y_mri_test, y_mri_pred)]]
linr_results_mri = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_mri

In [ ]:
linr_y_test = y_mri_test
linr_y_pred = y_mri_pred
linr_compare = pd.DataFrame(
    {'y_test_real_age': linr_y_test,
     'linr_y_pred_age': linr_y_pred,
    })
linr_compare_mri = linr_compare.reset_index()
#linr_compare_mri

In [ ]:
y_mri_pred = llreg.predict(X_mri_test)

In [ ]:
print('R2 score Lasso-linear regression: %.3f' % llreg.score(X_mri_test,y_mri_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_mri_test, y_mri_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_mri_test, y_mri_pred))

In [ ]:
data= [[
    'lasso regression',
    'unharm_mixed_llreg.sav',
    mean_absolute_error(y_mri_test, y_mri_pred),
    llreg.score(X_mri_test,y_mri_test),
    metrics.explained_variance_score(y_mri_test, y_mri_pred)]]
llreg_results_mri = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
#llreg_results_mri

In [ ]:
llreg_y_test = y_mri_test
llreg_y_pred = y_mri_pred
llreg_compare = pd.DataFrame(
    {'y_test_real_age': llreg_y_test,
     'llreg_y_pred_age': llreg_y_pred,
    })
llreg_compare_mri = llreg_compare.reset_index()
#llreg_compare_mri

In [ ]:
y_mri_pred = dtree.predict(X_mri_test)

In [ ]:
print('R2 score decision tree regression: %.3f' % dtree.score(X_mri_test,y_mri_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_mri_test, y_mri_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_mri_test, y_mri_pred))

In [ ]:
data= [[
    'decision tree',
    'unharm_mixed_dtree.sav',
    mean_absolute_error(y_mri_test, y_mri_pred),
    dtree.score(X_mri_test,y_mri_test),
    metrics.explained_variance_score(y_mri_test, y_mri_pred)]]
dtree_results_mri = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
dtree_results_mri

In [ ]:
dtree_y_test = y_mri_test
dtree_y_pred = y_mri_pred
dtree_compare = pd.DataFrame(
    {'y_test_real_age': dtree_y_test,
     'dtree_y_pred_age': dtree_y_pred,
    })
dtree_compare_mri = dtree_compare.reset_index()
#dtree_compare_mri

In [ ]:
y_mri_pred = regr.predict(X_mri_test)

In [ ]:
print('R2 score MLP regression: %.3f' % regr.score(X_mri_test,y_mri_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_mri_test, y_mri_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_mri_test, y_mri_pred))

In [ ]:
data= [[
    'multilayered perceptron',
    'unharm_mixed_regr.sav',
    mean_absolute_error(y_mri_test, y_mri_pred),
    regr.score(X_mri_test,y_mri_test),
    metrics.explained_variance_score(y_mri_test, y_mri_pred)]]
regr_results_mri = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
regr_results_mri

In [ ]:
regr_y_test = y_mri_test
regr_y_pred = y_mri_pred
regr_compare = pd.DataFrame(
    {'y_test_real_age': regr_y_test,
     'linr_y_pred_age': regr_y_pred,
    })
regr_compare_mri = regr_compare.reset_index()
#regr_compare_mri

In [ ]:
y_mri_pred = svr_p2.predict(X_mri_test)

In [ ]:
print('R2 score SVR poly2 regression: %.3f' % svr_p2.score(X_mri_test,y_mri_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_mri_test, y_mri_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_mri_test, y_mri_pred))

In [ ]:
data= [[
    'svr poly degree 2',
    'unharm_mixed_svrp2.sav',
    mean_absolute_error(y_mri_test, y_mri_pred),
    svr_p2.score(X_mri_test,y_mri_test),
    metrics.explained_variance_score(y_mri_test, y_mri_pred)]]
svrp2_results_mri = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
svrp2_results_mri

In [ ]:
svrp2_y_test = y_mri_test
svrp2_y_pred = y_mri_pred
svrp2_compare = pd.DataFrame(
    {'y_test_real_age': svrp2_y_test,
     'svrp2_y_pred_age': svrp2_y_pred,
    })
svrp2_compare_mri = svrp2_compare.reset_index()
#svrp2_compare_mri

In [ ]:
y_mri_pred = etreg.predict(X_mri_test)

In [ ]:
print('R2 score Extra tree regression: %.3f' % etreg.score(X_mri_test,y_mri_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_mri_test, y_mri_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_mri_test, y_mri_pred))

In [ ]:
data= [[
    'extra trees',
    'unharm_mixed_etreg.sav',
    mean_absolute_error(y_mri_test, y_mri_pred),
    etreg.score(X_mri_test,y_mri_test),
    metrics.explained_variance_score(y_mri_test, y_mri_pred)]]
etreg_results_mri = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_mri

In [ ]:
etreg_y_test = y_mri_test
etreg_y_pred = y_mri_pred
etreg_compare = pd.DataFrame(
    {'y_test_real_age': etreg_y_test,
     'etreg_y_pred_age': etreg_y_pred,
    })
etreg_compare_mri = etreg_compare.reset_index()
#etreg_compare_mri

In [ ]:
y_mri_pred = eregr.predict(X_mri_test)

In [ ]:
print('R2 score elasticnetCV: %.3f' % eregr.score(X_mri_test,y_mri_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_mri_test, y_mri_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_mri_test, y_mri_pred))

In [ ]:
data= [[
    'elasticnet CV',
    'unharm_mixed_eregr.sav',
    mean_absolute_error(y_mri_test, y_mri_pred),
    eregr.score(X_mri_test,y_mri_test),
    metrics.explained_variance_score(y_mri_test, y_mri_pred)]]
eregr_results_mri = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_mri

In [ ]:
eregr_y_test = y_mri_test
eregr_y_pred = y_mri_pred
eregr_compare = pd.DataFrame(
    {'y_test_real_age': eregr_y_test,
     'etreg_y_pred_age': eregr_y_pred,
    })
eregr_compare_mri = eregr_compare.reset_index()
#etreg_compare_mri

In [ ]:
# Agregate mixed results on MRI and optional save

In [ ]:
mixed_based_unharmonized_on_mri =pd.concat([linr_results_mri,
                   llreg_results_mri,
                   dtree_results_mri,
                   regr_results_mri,
                   svrp2_results_mri,
                   eregr_results_mri,
                   etreg_results_mri],
                   axis=0)
mixed_based_unharmonized_on_mri

In [ ]:
data_frames1 = [linr_compare_mri, llreg_compare_mri, dtree_compare_mri,]# regr_compare, ]#etreg_compare, svrp2_compare,]
real_versus_projected_y1 = reduce(lambda  left,right: pd.merge(left,right,on=["index"],
                                            how='outer'), data_frames1)
#real_versus_projected_y1
data_frames2 = [eregr_compare_mri, svrp2_compare_mri, etreg_compare_mri,]
real_versus_projected_y2 = reduce(lambda  left,right: pd.merge(left,right,on=["index"],
                                            how='outer'), data_frames2)
#real_versus_projected_y2
real_versus_projected_y2 = sep.drop_y(real_versus_projected_y2 )
real_versus_projected_y1 = sep.drop_y(real_versus_projected_y1 )
#real_versus_projected_y1

In [ ]:
data_frames3 = [real_versus_projected_y1, real_versus_projected_y2,]
real_versus_projected_y3_mixed_on_mri = reduce(lambda  left,right: pd.merge(left,right,on=["index"],
                                            how='outer'), data_frames3)
real_versus_projected_y3_mixed_on_mri.head(3)

In [ ]:
# # optionally save of csvs of algorithms and results
# mixed_based_unharmonized_on_mri.to_csv('mixed_based_unharmonized_on_mri.csv')
# real_versus_projected_y3_mixed_on_mri.to_csv('real_versus_projected_y3_mixed_on_mri.csv')

# Running mixed model over SABRE dataset

In [ ]:
sabre_ml_matrix = SABRE.drop('participant_id', axis=1)
X_sabre = sabre_ml_matrix.drop('age', axis =1)
X_sabre = X_sabre.values
X_sabre = X_sabre.astype('float')
y_sabre = sabre_ml_matrix['age'].values
y_sabre=y_sabre.astype('float')

In [ ]:
X_sabre_test = X_sabre
y_sabre_test = y_sabre

In [ ]:
y_sabre_pred = linr.predict(X_sabre_test)

In [ ]:
print('R2 score Linear regression: %.3f' % linr.score(X_sabre_test,y_sabre_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_sabre_test, y_sabre_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_sabre_test, y_sabre_pred))

In [ ]:
data= [[
    'Linear Reg',
    'unharm_mix_linr.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    linr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
linr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_sabre

In [ ]:
y_sabre_pred = llreg.predict(X_sabre_test)

In [ ]:
print('R2 score Lasso regression: %.3f' % llreg.score(X_sabre_test,y_sabre_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_sabre_test, y_sabre_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_sabre_test, y_sabre_pred))

In [ ]:
data= [[
    'Lasso',
    'unharm_mix_lassor.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    llreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
llreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_sabre

In [ ]:
y_sabre_pred = dtree.predict(X_sabre_test)

In [ ]:
print('R2 score Decision tree: %.3f' % dtree.score(X_sabre_test,y_sabre_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_sabre_test, y_sabre_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_sabre_test, y_sabre_pred))

In [ ]:
data= [[
    'Decision tree',
    'unharm_mri_svrp2.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    dtree.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
dtree_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
dtree_results_sabre

In [ ]:
y_sabre_pred = regr.predict(X_sabre_test)

In [ ]:
print('R2 score MLP regression: %.3f' % regr.score(X_sabre_test,y_sabre_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_sabre_test, y_sabre_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_sabre_test, y_sabre_pred))

In [ ]:
data= [[
    'MLP regression',
    'unharm_mri_regr.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    regr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
regr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
regr_results_sabre

In [ ]:
y_sabre_pred = svr_p2.predict(X_sabre_test)

In [ ]:
print('R2 score SVR polynomial regression: %.3f' % svr_p2.score(X_sabre_test,y_sabre_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_sabre_test, y_sabre_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_sabre_test, y_sabre_pred))

In [ ]:
data= [[
    'Svr P2',
    'unharm_mri_svrp2.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    svr_p2.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
svr_p2_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
svr_p2_results_sabre

In [ ]:
y_sabre_pred = eregr.predict(X_sabre_test)

In [ ]:
print('R2 score ElasticNet CV : %.3f' % eregr.score(X_sabre_test,y_sabre_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_sabre_test, y_sabre_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_sabre_test, y_sabre_pred))

In [ ]:
data= [[
    'ElasticnetCV',
    'unharm_mri_elasticregr.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    eregr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
eregr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_sabre

In [ ]:
y_sabre_pred = etreg.predict(X_sabre_test)

In [ ]:
print('R2 score extra tree regression: %.3f' % etreg.score(X_sabre_test,y_sabre_test))
print('Explained variance score: %.3f'  % metrics.explained_variance_score(y_sabre_test, y_sabre_pred))
print('The mean absolute error: %.3f' % mean_absolute_error(y_sabre_test, y_sabre_pred))

In [ ]:
data= [[
    'Extra trees',
    'unharm_mri_etreg.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    etreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
etreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_sabre

In [ ]:
mix_based_unharmonized_on_sabre =pd.concat([linr_results_sabre,
                   llreg_results_sabre,
                   dtree_results_sabre,
                   regr_results_sabre,
                   svr_p2_results_sabre,
                   eregr_results_sabre,
                  etreg_results_sabre],
                  axis=0)
mix_based_unharmonized_on_sabre